Импорт библиотек

In [ ]:
import pandas as pd
import seaborn as sns
from sklearn.base import BaseEstimator
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split 
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import os
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv(os.getcwd() + r'\input\fish_train (1).csv')

In [ ]:
data['Species'].value_counts()

Всего датасет содержит 159 записей. Мы предварительно зарезервировали часть датасета для задания по преодолению бейслайна (baseline). В настоящем задании подразумевается работа с датасетом, содержащим 119 записей.

# 0. Применение полиномиальной регрессии для предсказания непрерывного параметра

Представленный набор данных — это набор данных о семи различных типах рыб, продаваемых в некоторой рыбной лавке. Наша цель заключается в том, чтобы предсказать массу рыбы по каким-то косвенным признакам, известным о рыбе. Сами признаки, быть может, нужно синтезировать из тех, что известны.

При помощи <code>train_test_split()</code> разбейте набор данных на обучающую и тестовую выборки с параметрами, указанными в вашем задании. Используйте стратификацию по колонке <code>Species</code>. Стратификация позволит сохранить доли представленных объектов (по представителям типов рыб) в тренировочной и тестовой выборках.

In [ ]:
data

In [ ]:
print(data['Species'])

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(data[['Species', 'Length1', 'Length2','Length3', 'Height','Width' ]],data[['Weight']], test_size=0.2, random_state=35, stratify=data['Species'])

Вычислите выборочное среднее колонки <code>Width</code> полученной тренировочной выборки.

In [ ]:
print(train_x['Width'].mean())

# 1. Построение базовой модели

Избавьтесь от категориальных признаков и обучите модель линейной регрессии (<code>LinearRegression()</code>) на тренировочном наборе данных. Выполните предсказания для тестового набора данных. Оцените модель при помощи метрики <code>r2_score()</code>.

In [ ]:
train = train_x[['Species','Length1', 'Length2','Length3', 'Height','Width' ]]
train['Weight'] = train_y[['Weight']]
sns.pairplot(train[['Length1', 'Length2','Length3', 'Height','Width' ]])
test = test_x[['Length1', 'Length2','Length3', 'Height','Width' ]]
test['Weight'] = test_y[['Weight']]


In [ ]:
lr = LinearRegression()
lr.fit(train_x[['Length1', 'Length2','Length3', 'Height','Width' ]], train_y)
y_pred = lr.predict(test_x[['Length1', 'Length2','Length3', 'Height','Width' ]])
print(lr.score(test_x[['Length1', 'Length2','Length3', 'Height','Width' ]], test_y))

# 2. Добавление предварительной обработки признаков.

## Использование PCA

При помощи, например, <code>sns.heatmap()</code>, постройте матрицу корреляций признаков тренировочного набора данных и определите тройку наиболее коррелированных между собой признаков.

In [ ]:
corr_train = train[['Length1', 'Length2','Length3', 'Height','Width' ]].corr()
print(corr_train)
sns.heatmap(corr_train)

In [ ]:
pca = PCA(n_components=1, svd_solver='full')
pca.fit_transform(train_x[['Length1', 'Length2','Length3']])
print(pca.singular_values_)


In [ ]:
print(pca.explained_variance_ratio_)


In [ ]:
train_x_pca = train_x[[ 'Height','Width' ]]
train_x_pca['Length'] = pca.transform(train_x[[ 'Length1', 'Length2','Length3' ]])

In [ ]:
train_x_pca['Length']

In [ ]:

test_x_pca = test_x[[ 'Height','Width' ]]
test_x_pca['Length']=pca.transform(test_x[['Length1', 'Length2','Length3']])

Линейные модели достаточно плохо реагируют на коррелированные признаки, поэтому от таких признаков имеет смысл избавиться еще до начала обучения.

Для уменьшения количества неинформативных измерений используйте метод главных компонент. 

1) Примените метод главных компонент (<code>PCA(n_components=3, svd_solver='full')</code>) для трех найденных наиболее коррелированных признаков. 

2) Вычислите долю объясненной дисперсии при использовании только первой главной компоненты. 

3) Замените три наиболее коррелированных признака на новый признак <code>Lengths</code>, значения которого совпадают со значениями счетов первой главной компоненты.

In [ ]:
lr.fit(train_x_pca, train_y)

lr.score(test_x_pca, test_y)


Примените полученное преобразование для тех же признаков в тестовом наборе данных. Обратите внимание, что заново обучать преобразование `PCA` не нужно. Аналогично предыдущему этапу замените три рассмотренных признака на один.

In [ ]:
# < ENTER YOUR CODE HERE > 

Обучите базовую модель линейной регресси на полученных тренировочных данных, снова выбросив категориальные признаки. Выполните предсказания для тестовых данных, оцените при помощи <code>r2_score()</code>.

In [ ]:
# < ENTER YOUR CODE HERE > 

Видно, что точность значительно не изменилась.

## Модификация признаков

Постройте графики зависимостей признаков от целевой переменной, например, при помощи <code>sns.pairplot()</code>.

In [ ]:
# < ENTER YOUR CODE HERE > 

Видно, что масса, вообще говоря, нелинейно зависит от остальных параметров. Значит, чтобы линейная модель хорошо справлялась с предсказанием, признаки имеет смысл преобразовать так, чтобы зависимость стала более похожей на линейную. Но как придумать такую зависимость?

Логично предположить, что масса рыбы должна каким-то гладким образом зависеть от остальных параметров, отвечающих так или иначе за размеры. Если впомнить, что масса — это произведение плотности на объем, то

$$
m = \rho \cdot V.
$$

Допустим, что средняя плотность у всех рыб одинаковая, и вспомним, что при гомотетии объем объекта зависит от линейных размеров как куб, тогда получим

$$
m\sim V\sim d^3
$$

Все признаки тренировочного и тестового наборов данных, отвечающие так или иначе за размеры (<code>Height, Width, Lengths</code>), возведите в третью степень, и проверьте, стала ли зависимость массы от этих признаков похожа на линейную.

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
class ColCube(BaseEstimator, TransformerMixin):
    def __init__(self, cols):
        self.cols = cols


    def fit(self, X=None, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        try:
            for col in self.cols:
                X[col] = X[col].astype(float).apply(lambda x : x ** 3)
            return X
        except Exception as e:
            print(f'{self.cols} не могут быть возведены в степень {e}')
            return X



In [ ]:
from sklearn.compose import ColumnTransformer
ct = ColumnTransformer([
    ('cube', ColCube(['Length', 'Height', 'Width']), ['Length', 'Height', 'Width'])
])

In [ ]:
train_x_pca_cube = ct.fit_transform(train_x_pca)

In [ ]:
train_x_pca_cube = pd.DataFrame(train_x_pca_cube)
train_x_pca_cube.rename(columns={0: 'Length', 1: 'Height', 2: 'Width'}, inplace=True)
test_x_pca_cube = ct.transform(test_x_pca)
test_x_pca_cube = pd.DataFrame(test_x_pca_cube)
test_x_pca_cube.rename(columns={0: 'Length', 1: 'Height', 2: 'Width'}, inplace=True)



Введите выборочное среднее колонки <code>Width</code> тренировочного набора данных после возведения в куб.

In [ ]:
print(train_x_pca_cube['Width'].mean())
print(train['Width'].mean())

Выберите изображения, соответствующие зависимости <code>Weight</code> от <code>Width</code> до преобразования и после.

Обучите базовую модель линейной регресси на полученных тренировочных данных, снова выбросив категориальные признаки. Выполните предсказания для тестовых данных, оцените при помощи `r2_score()`.

In [ ]:
lr.fit(train_x_pca_cube, train_y)
lr.score(test_x_pca_cube, test_y)

Обратите внимание на то, как такая нехитрая работа с признаками помогла разительно улучшить точность модели!

## Добавление категориальных признаков

Произведите <code>one-hot</code> кодировние категориального признака `Species`, например, с помощью <code>pd.get_dummies()</code>.

Обучите модель линейной регресси на полученных тренировочных данных. Выполните предсказания для тестовых данных, оцените модель при помощи <code>r2_score()</code>.

<b>Примечание</b>: Мы специально использовали стратифицированное разделение, чтобы все значения категориального признака <code>Species</code> присутствовали во всех наборах данных. Но такое возможно не всегда. Про то, как с этим бороться можно почитать, [например, здесь](https://predictivehacks.com/?all-tips=how-to-deal-with-get_dummies-in-train-and-test-dataset).

In [ ]:
train_x_pca_cube_categorical = train_x_pca_cube.copy()
train_x_pca_cube_categorical['Species'] = train_x['Species']
test_x_pca_cube_categorical = test_x_pca_cube.copy()
test_x_pca_cube_categorical['Species'] = test_x['Species']

In [ ]:
train_x_pca_cube_categorical

In [ ]:
test_x_pca_cube_categorical

И снова точность возрасла.

Как можно увидеть, после `one-hot` кодирования признаки стали коррелированы. От этого можно избавиться, например, при помощи параметра `drop_first=True`. Заново обучите модель после исправления этого недочета. Выполните предсказания для тестовых данных, оцените модель при помощи <code>r2_score()</code>.

In [ ]:
train_x_pca_cube_categorical = pd.concat([train_x_pca_cube_categorical, pd.get_dummies(train_x_pca_cube_categorical['Species'])], axis=1) 

In [ ]:
train_x_pca_cube_categorical = train_x_pca_cube_categorical.drop(['Species'], axis=1)

In [ ]:
test_x_pca_cube_categorical = pd.concat([test_x_pca_cube_categorical, pd.get_dummies(test_x_pca_cube_categorical['Species'])], axis=1)

In [ ]:
test_x_pca_cube_categorical = test_x_pca_cube_categorical.drop(['Species'], axis=1)

На таком сравнительно небольшом наборе данных, впрочем, разницы мы не видим.

#### Я хз поч, но дурачки решили, что лучше кодировать до разделения на выборки. То есть они подразумевали, что так надо:  

dummies = pd.get_dummies(df['Species'])

df[list(dummies.columns)] = dummies 

со всем датасетом и потом его делить

In [ ]:
train_x_pca_cube_categorical

In [ ]:
test_x_pca_cube_categorical['Parkki'] = False
test_x_pca_cube_categorical['Pike'] = False
test_x_pca_cube_categorical['Whitefish'] = False

In [ ]:
train_x_pca_cube_categorical.columns

In [ ]:
test_x_pca_cube_categorical = test_x_pca_cube_categorical[['Length', 'Height', 'Width', 'Bream', 'Parkki', 'Perch', 'Pike',
       'Roach', 'Smelt', 'Whitefish']]

In [ ]:
lr.fit(train_x_pca_cube_categorical, train_y)
lr.score(test_x_pca_cube_categorical, test_y)

In [ ]:
train_x_pca_cube_categorical = train_x_pca_cube_categorical.drop(['Bream', 'Parkki', 'Perch', 'Pike',
       'Roach', 'Smelt', 'Whitefish' ], axis=1)
test_x_pca_cube_categorical = test_x_pca_cube_categorical.drop(['Bream', 'Parkki', 'Perch', 'Pike',
       'Roach', 'Smelt', 'Whitefish'], axis=1)

In [ ]:
train_x_pca_cube_categorical

In [ ]:
test_x_pca_cube_categorical

In [ ]:
train_x_pca_cube_categorical['Species'] = train_x['Species']
test_x_pca_cube_categorical['Species'] = test_x['Species']

In [ ]:
train_x_pca_cube_categorical = pd.concat(
    [train_x_pca_cube_categorical, pd.get_dummies(train_x_pca_cube_categorical['Species'], drop_first=True)], axis=1)
train_x_pca_cube_categorical = train_x_pca_cube_categorical.drop(['Species'], axis=1)
test_x_pca_cube_categorical = pd.concat(
    [test_x_pca_cube_categorical, pd.get_dummies(test_x_pca_cube_categorical['Species'], drop_first=True)], axis=1)
test_x_pca_cube_categorical = test_x_pca_cube_categorical.drop(['Species'], axis=1)

In [ ]:
train_x_pca_cube_categorical

In [ ]:
test_x_pca_cube_categorical

In [ ]:
test_x_pca_cube_categorical['Parkki'] = False
test_x_pca_cube_categorical['Pike'] = False
test_x_pca_cube_categorical['Whitefish'] = False

In [ ]:
test_x_pca_cube_categorical = test_x_pca_cube_categorical[['Length', 'Height', 'Width', 'Parkki', 'Perch', 'Pike',
       'Roach', 'Smelt', 'Whitefish']]

In [ ]:
lr.fit(train_x_pca_cube_categorical, train_y)
lr.score(test_x_pca_cube_categorical, test_y)

## Вторая часть задания: сделаю так, как хотят с кодировкой

In [1]:
import pandas as pd
import seaborn as sns
from sklearn.base import BaseEstimator
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split 
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import os
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.base import BaseEstimator, TransformerMixin

In [2]:
data = pd.read_csv(os.getcwd() + r'\input\fish_train (1).csv')

In [3]:
train = data.copy()


In [7]:
class PCA_transformer(BaseEstimator, TransformerMixin):
    def __init__(self, n_components, svd_solver='full'):
        
        self.n_components = n_components
        self.svd_solver = svd_solver
        self.pca = PCA(n_components=self.n_components, svd_solver=self.svd_solver)
        self.cols =  ['Length1', 'Length2', 'Length3']
        
    def fit(self, X, y=None):
        self.pca.fit(X[self.cols])
        return self
    
    def transform(self, X, y=None):
        X = X.copy()
        X['Length'] = self.pca.transform(X[self.cols])
        X = X.drop(self.cols, axis=1)
        print('Выполнена трансформация в PCAtransformer')

        return X
    
        

In [38]:
from sklearn.base import BaseEstimator, TransformerMixin
class ColCube(BaseEstimator, TransformerMixin):
    def __init__(self, ):
        self.cols = ['Length', 'Height', 'Width']


    def fit(self, X=None, y=None):
        return self

    def transform(self, X, y=None):
        X = X.copy()
        try:
            for col in self.cols:
                X[col] = X[col].astype(float).apply(lambda x : x ** 3)
            print('Выполнена трансформация в ColCube')
            return X
        except Exception as e:
            print(f'{self.cols} не могут быть возведены в степень {e}')
            return X

In [39]:
class Cat(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        self.dummies = pd.get_dummies(X['Species'])
        return self
    
    def transform(self, X, y=None):
        try:
            X[list(self.dummies.columns)] = self.dummies
            print('Выполнена трансформация в Cat')

            return X.drop('Species', axis=1)

        except Exception as e:
            print(f'Не удалось выполнить кодирование категориальных признаков {e}')
            return X.drop('Species', axis=1)
        


In [28]:
from sklearn.pipeline import make_pipeline, Pipeline

pipeline = Pipeline(
    steps=[
        ('pca', PCA_transformer(n_components=1, svd_solver='full')),
        ('ct', ColCube()),
        ('cat', Cat()),
        ('lr', LinearRegression()),
        
    ])

In [29]:
train_x, train_y = train[['Species', 'Length1', 'Length2','Length3', 'Height','Width' ]], train['Weight']

In [30]:
train_x 

,Species,Length1,Length2,Length3,Height,Width
0,Roach,20.4,22.0,24.7,5.8045,3.7544
1,Perch,25.4,27.5,28.9,7.2828,4.5662
2,Bream,26.5,29.0,34.0,12.4440,5.1340
3,Bream,36.2,39.5,45.3,18.7542,6.7497
4,Perch,19.0,21.0,22.5,5.6925,3.5550
...,...,...,...,...,...,...
114,Parkki,23.0,25.0,28.0,11.0880,4.1440
115,Pike,42.0,45.0,48.0,6.9600,4.8960
116,Whitefish,37.3,40.0,43.5,12.3540,6.5250
117,Perch,20.0,22.0,23.5,6.1100,3.4075


In [31]:
train_y

0       150.0
1       250.0
2       430.0
3       925.0
4       110.0
        ...  
114     273.0
115     500.0
116    1000.0
117     120.0
118     820.0
Name: Weight, Length: 119, dtype: float64

In [32]:
pipeline.fit(train_x, train_y)

Выполнена трансформация в PCAtransformer
Выполнена трансформация в ColCube
Выполнена трансформация в Cat


,steps,"[('pca', ...), ('ct', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,n_components,1
,svd_solver,'full'
,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [34]:
test = pd.read_csv(os.getcwd() + r'\input\fish_reserved.csv')
test_x = test[['Species', 'Length1', 'Length2','Length3', 'Height','Width' ]]

In [35]:
test_x

,Species,Length1,Length2,Length3,Height,Width
0,Roach,19.4,21.0,23.7,6.1146,3.2943
1,Perch,19.3,21.3,22.8,6.3840,3.5340
2,Perch,17.2,19.0,20.2,5.6358,3.0502
3,Parkki,17.5,19.0,21.3,8.3922,2.9181
4,Parkki,16.3,17.7,19.8,7.4052,2.6730
5,Perch,22.0,24.0,25.5,7.2930,3.7230
6,Perch,15.0,16.2,17.2,4.5924,2.6316
7,Pike,40.0,42.5,45.5,6.8250,4.4590
8,Bream,27.6,30.0,35.0,12.6700,4.6900
9,Roach,18.6,20.0,22.2,6.2160,3.5742


In [36]:
y_calc = list(pipeline.predict(test_x))

Выполнена трансформация в PCAtransformer
Выполнена трансформация в ColCube
Выполнена трансформация в Cat


In [37]:
print([float(y) for y in y_calc])

[97.04751736579752, 147.01591527512176, 46.24886505504912, 93.29266727207855, 139.6809624956793, 162.17334371664066, 110.4709377769276, 269.02822495562816, 489.2751781974207, 85.29930405125216, 1625.7105951999229, 60.60842759890548, 691.1582806373809, 961.4252786826861, 104.73942377100235, 651.8070344040914, 21.17464594737018, 902.1495707393716, 1184.4513599184675, 28.150774191410704, 577.6704620759928, 205.14840119802147, 29.73035618473253, 131.9983054612747, 143.33617517017785, 558.0253681114427, 220.62831176107557, 91.39982565234368, 64.68410550404442, 25.117476216289447, 972.4589295683655, 634.3500488141937, 591.1455565590988, 340.29533938045944, 74.61736790542733, 224.25379234481045, 34.852917210509645, 952.1866298512133, 768.1527816497411, 133.0811699974103]


### Вот это эталонное значение, сделано нейронкой на основании моего кода

In [40]:
import os
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.decomposition import PCA
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, OneHotEncoder
from sklearn.linear_model import Ridge
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error

# Custom PCA для lengths (с возвратом DataFrame)
class PCA_transformer(BaseEstimator, TransformerMixin):
    def __init__(self, n_components=2, svd_solver='full'):  # Default 2 для большего variance
        self.n_components = n_components
        self.svd_solver = svd_solver
        self.pca = PCA(n_components=self.n_components, svd_solver=self.svd_solver)
        self.cols = ['Length1', 'Length2', 'Length3']
    
    def fit(self, X, y=None):
        self.pca.fit(X[self.cols])
        return self
    
    def transform(self, X, y=None):
        X = X.copy()
        pca_components = self.pca.transform(X[self.cols])
        # Добавляем компоненты как новые колонки (PCA_1, PCA_2, ...)
        for i in range(self.n_components):
            X[f'PCA_Length_{i+1}'] = pca_components[:, i]
        X = X.drop(self.cols, axis=1)
        return X

# Загрузка данных
data = pd.read_csv(os.getcwd() + r'\input\fish_train (1).csv')
train = data.copy()

# Разделим на фичи и таргет
features = ['Species', 'Length1', 'Length2', 'Length3', 'Height', 'Width']
X = train[features]
y = train['Weight']

# Локальный сплит для валидации (80/20)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Пайплайн для числовых фич (PCA на lengths, poly на all numerics после, scaler)
numeric_features = ['Length1', 'Length2', 'Length3', 'Height', 'Width']
numeric_transformer = Pipeline(steps=[
    ('pca', PCA_transformer()),  # PCA только на lengths, добавит PCA_Length_*
    ('poly', PolynomialFeatures(include_bias=False)),  # Poly на оставшихся + PCA
    ('scaler', StandardScaler())
])

# Пайплайн для категориальных
categorical_features = ['Species']
categorical_transformer = OneHotEncoder(sparse_output=False, handle_unknown='ignore', drop='first')

# ColumnTransformer для комбо
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Полный пайплайн с моделью
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', Ridge(random_state=42))
])

# GridSearch для тюнинга
param_grid = {
    'preprocessor__num__pca__n_components': [1, 2, 3],  # Тюним PCA
    'preprocessor__num__poly__degree': [2, 3],  # Degree 3 для cubing + interactions
    'regressor__alpha': [0.01, 0.1, 1, 10, 100]  # Регуляризация
}

grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='r2', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Лучшие параметры и CV-score
print("Best parameters:", grid_search.best_params_)
print("Best CV R²:", grid_search.best_score_)

# Валидация на локальном val
y_val_pred = grid_search.predict(X_val)
print("Validation R²:", r2_score(y_val, y_val_pred))
print("Validation MSE:", mean_squared_error(y_val, y_val_pred))

# Предикт на reserved.csv
test = pd.read_csv(os.getcwd() + r'\input\fish_reserved.csv')
test_x = test[features]
y_calc = grid_search.predict(test_x)
print("Predicted weights on reserved:", [float(y) for y in y_calc])

Best parameters: {'preprocessor__num__pca__n_components': 1, 'preprocessor__num__poly__degree': 2, 'regressor__alpha': 10}
Best CV R²: 0.9667577680278541
Validation R²: 0.9728159632152688
Validation MSE: 2283.404721843857
Predicted weights on reserved: [100.13834393143446, 133.14266332990434, 90.21868233508633, 102.47702564357036, 75.55736717043112, 176.06776466241178, 59.49268906967478, 500.9085315507496, 425.4170333919375, 108.91966206318125, 1703.6741355468034, 171.26042268468944, 651.5787353969066, 1051.8107897957257, 36.31893619727316, 642.3039811380122, 116.79038478478776, 813.1579101230075, 1328.9990222547374, 87.30798679145539, 609.5443152643448, 185.62629245860828, 87.97238820626131, 169.4784193747849, 324.1836650356098, 599.929846736123, 317.51546232217726, 8.3112169048066, 12.322663307531968, 13.551569226316303, 1005.730720928115, 498.29235462076286, 561.5902428550764, 151.0984688857479, 2.5517012216802186, 65.32621036231313, 131.36174237992805, 908.391760863213, 758.4893581